# Lab 4: Introduction to Mining Sequence Data

In this lab, we will first focus on Sequence Comparison/Alignment. We will compute the Levenshtein Edit Distance for country names and implement a simple version of BLAST to compare genomes. Then, we will turn to Sequence Prediction/Labeling and will walk through a Named Entity Extraction task.

In [4]:
# %pip install numpy --upgrade
# # and then restart kernel
# %pip install country_list
from country_list import countries_for_language
# %pip install Bio
from Bio.Blast import NCBIWWW, NCBIXML # calling online version
#%pip install sklearn
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

# %pip install pomegranate
# %pip install pomegranate --no-cache-dir --no-binary :all:
# may need to run 'pip install pomegranate --no-cache-dir --no-binary :all:' depending on setup
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
# if when running import, give an error saying something about numpy version:
# %pip install sklearn_crfsuite
import sklearn_crfsuite

In [5]:
import numpy as np
import pandas as pd
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter, defaultdict

## 1. Sequence Comparison/Alignment

### 1.1 Levenshtein Edit Distance

Levenshtein Edit Distance helps us assess the similarity of sequences. It is calculated in terms of the minimum amount of operations (insertion, deletions and substitutions) that must be made to make two sequences identical. The smaller the distance, the greater the similarity between the sequences. To show how it is computed, we can generate a matrix and find the "trace" or the path for the minimum edit distance.

In [7]:
# source: https://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/

def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros((size_x, size_y), dtype="object") # changed dtype
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    
    # added code for readability
    matrix[0,0] = ""
    for x in range(1, size_x):
        matrix [x, 0] = seq1[x-1]
    for y in range(1, size_y):
        matrix [0, y] = seq2[y-1]
        
    print(matrix)
    
    return matrix[size_x - 1, size_y - 1]

First, to test this function, let's try out the example from class to compute the distance between "William Cohen" and "William Cohon".

In [8]:
print("Edit Distance =", levenshtein("William Cohen", "Willliam Cohon"))

[['' 'W' 'i' 'l' 'l' 'l' 'i' 'a' 'm' ' ' 'C' 'o' 'h' 'o' 'n']
 ['W' 0 1 2 3 4 5 6 7 8 9 10 11 12 13]
 ['i' 1 0 1 2 3 4 5 6 7 8 9 10 11 12]
 ['l' 2 1 0 1 2 3 4 5 6 7 8 9 10 11]
 ['l' 3 2 1 0 1 2 3 4 5 6 7 8 9 10]
 ['i' 4 3 2 1 1 1 2 3 4 5 6 7 8 9]
 ['a' 5 4 3 2 2 2 1 2 3 4 5 6 7 8]
 ['m' 6 5 4 3 3 3 2 1 2 3 4 5 6 7]
 [' ' 7 6 5 4 4 4 3 2 1 2 3 4 5 6]
 ['C' 8 7 6 5 5 5 4 3 2 1 2 3 4 5]
 ['o' 9 8 7 6 6 6 5 4 3 2 1 2 3 4]
 ['h' 10 9 8 7 7 7 6 5 4 3 2 1 2 3]
 ['e' 11 10 9 8 8 8 7 6 5 4 3 2 2 3]
 ['n' 12 11 10 9 9 9 8 7 6 5 4 3 3 2]]
Edit Distance = 2


We see that it matches the result from class. Next, let's try an example with countries. First, let's comment out the print command from the function above to reduce the size of our output.

In [9]:
# source: https://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros((size_x, size_y), dtype="object") # changed dtype
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    
    # added code for readability
    matrix[0,0] = ""
    for x in range(1, size_x):
        matrix [x, 0] = seq1[x-1]
    for y in range(1, size_y):
        matrix [0, y] = seq2[y-1]
        
    #print(matrix)
    
    return matrix[size_x - 1, size_y - 1]

In order for this to work, we must also load a list of countries for reference using the country_list module.

In [10]:
countries = list(dict(countries_for_language('en')).values())
print(countries)

['Afghanistan', 'Åland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua & Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia & Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Caribbean Netherlands', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo - Brazzaville', 'Congo - Kinshasa', 'Cook Islands', 'Costa Rica', 'Côte d’Ivoire', 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Esw

Looking at the country names, it is evident that some of them may be difficult to spell correctly. Therefore, one way we can use the Levenshtein Distance is to find the correct spelling of a country name by finding the one that is closest our misspelling. See below for an example.

In [11]:
# function to calculatedisrance
def find_closest_country(country_mispelling):
    distances = {}
    for country in countries:
        distance = levenshtein(country_mispelling, country)
        distances[country] = distance
    return sorted(distances.items(), key=lambda c:c[1])[0:3] # return the three closest countries

find_closest_country("Phillipines")

[('Philippines', 2), ('Chile', 7), ('Maldives', 7)]

In [12]:
### YOUR CODE: try this with other common country mispellings

### for inspiration: https://www.funtrivia.com/playquiz/quiz32051824b17d8.html

print(find_closest_country('Uragauy'))
print(find_closest_country('Mauritania'))

[('Uruguay', 3), ('Hungary', 4), ('Palau', 4)]
[('Mauritania', 0), ('Mauritius', 4), ('Tanzania', 5)]


In addition to helping identify the correct spelling of a word, Levenshtein Distance can help us with auto-completion tasks. See below for an example.

In [13]:
find_closest_country("Sri La")

[('Sri Lanka', 3), ('Syria', 3), ('Aruba', 4)]

### 1.2 BLAST: Basic Local Alignment Search Tool

We can think of BLAST as a heuristic model for performing local alignment tasks that operates by searching for high scoring segment pairs (HSPs). As discussed in lecture, BLAST views sequences as sequences of short words (k-tuple). Therefore, in the case of good alignment, we should see close matches between many of these sub-sequences, or HSPs. To showcase BLAST, we will utilize an online version provided through NCBI.

In [14]:
# sources: 
# https://www.tutorialspoint.com/biopython/biopython_overview_of_blast.htm
# https://blast.ncbi.nlm.nih.gov/Blast.cgi

help(NCBIWWW.qblast) # help documentation

Help on function qblast in module Bio.Blast.NCBIWWW:

qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, short_query=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_high=None, format_entrez_query=None, format_object=None, format_type='XML', ncbi_gi=None, results_file=None, show_overview=None, megablast=None, template_type=None, template_length=None)
    BLAST search using NCBI's

In [15]:
# example running qblast

# WARNING: CAN TAKE ~5-10 minutes to run depending on load

# example of poliovirus
# blastn = nucleotide search program
# nt = nucleotide database 
# sequence = GI number of the sequence (poliovirus in this case) = 12408699 

result_handle = NCBIWWW.qblast("blastn", "nt", sequence = 12408699) 
result_handle 

In [16]:
# write the results to a file

with open('polio_results.xml', 'w') as results_file: 
    results = result_handle.read() 
    results_file.write(results)

In [17]:
# look through the results

threshold = 1e-20 # think of this like a p-value
for record in NCBIXML.parse(open("polio_results.xml")): 
     if record.alignments: 
        for align in record.alignments: 
            for hsp in align.hsps: 
                if hsp.expect < threshold: 
                    print("Match:", align.title, "\n")

Match: gi|61252|emb|V01149.1| Human poliovirus 1 Mahoney, complete genome 

Match: gi|1012282727|gb|KU866422.1| Poliovirus 1 strain Mahoney_CDC, complete genome 

Match: gi|27085396|gb|AY184219.1| Human poliovirus 1 strain Sabin 1, complete genome 

Match: gi|283137719|gb|GQ984141.1| Human poliovirus 1 strain Sabin 1 isolate S302, complete genome 

Match: gi|61236|emb|V01148.1| Genome of human poliovirus type 1 (Mahoney strain). (One of two versions.) 

Match: gi|61257|emb|V01150.1| Human poliovirus strain Sabin 1 complete genome, strain Sabin 1 

Match: gi|987390362|gb|KT353719.1| Poliovirus 1 strain 1-B2, complete genome 

Match: gi|18644085|emb|AJ430385.1| Human poliovirus 1 genomic RNA for polyprotein gene, strain Cox 

Match: gi|193245070|gb|EU794954.1| Human poliovirus 1 isolate A49 polyprotein gene, complete cds 

Match: gi|643433715|gb|KJ170502.1| Human poliovirus 1 strain NIE0918388, complete genome >gi|643433717|gb|KJ170503.1| Human poliovirus 1 strain NIE0918389, complete ge

From running this query, we see that we get back a bunch of different samples of poliovirus variations.

In [18]:
### YOUR CODE: try this out with another nucleotide sequence

### inspiration: find the GI number of a sequence using https://www.ncbi.nlm.nih.gov/nuccore

COVID = 1798172431
result_handle = NCBIWWW.qblast('blastn', 'nt', sequence=1798172431)

## 2. Sequence Prediction/Labeling

For this part of the lab, we will be experimenting with different models to perform Named Entity Recognition (NER). The goal of NER is to label items into a set of predefined states/entities, such as an organizations, dates/times, geographical locations, etc. To do so, we will use a dummy majority classifier as our baseline and then will experiment with two sequence models introduced in lecture: Hidden Markov Models (HMMs) and Conditional Random Fields (CRFs).

### 2.1 Data Exploration, Data Preparation & Train-Test Splitting

In [19]:
# load the data

# data source: https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus
ner_dataset = pd.read_csv('ner_dataset.csv', encoding = "ISO-8859-1")
print(ner_dataset.shape)
ner_dataset.head()

(1048575, 4)


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [20]:
# how many sentences?
print("Number of sentences:", ner_dataset['Sentence #'].nunique())

# distribution of tags?
ner_dataset['Tag'].value_counts()

Number of sentences: 47959


O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: Tag, dtype: int64

Some notes about the data:

We can think of 'O' as background or not a named entity. We see that 'O' is by far the most common state indicating that most text is not a named entity in this case.

'B-' indicates the beginning of the named entity and 'I' indicates that it is a continued part of the named entity.

Overall, we see that the distribution of the named entities is heavily skewed with some entities appearing much more than others in the data. This is important to keep in mind when fitting models and assessing performance.


Next, we will want to fill in the NaN values in the 'Sentence' column with the previous value so that we can attribute each word/tag to the correct sentence.

In [21]:
# fill in NaN sentences
ner_dataset.fillna(method='ffill', inplace=True)
ner_dataset.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


As this dataset is very large, we will only work with 2,000 sentences in our training set and 1,000 sentences in our testing set to reduce computational costs.

In [22]:
ner_train = ner_dataset[0:44396] # first 2000 sentences
ner_test = ner_dataset[44396:66174] # next 1000 sentences

ner_X_train = ner_train.drop(columns="Tag") # training features
ner_y_train = ner_train.Tag # training outcome variable
ner_X_test = ner_test.drop(columns="Tag") # testing features
ner_y_test = ner_test.Tag # testing outcome variable

# checks
print(ner_train.shape)
print(ner_X_train.shape)
print(ner_y_train.shape)
print(ner_test.shape)
print(ner_X_test.shape)
print(ner_y_test.shape)

(44396, 4)
(44396, 3)
(44396,)
(21778, 4)
(21778, 3)
(21778,)


### 2.2 Model Fitting

#### 2.2.1 Dummy Majority Classifier

The first model we will implement is a dummy classifier that will label each word with the most frequent entity, which is 'O' in this case. This will serve as our baseline.

In [23]:
# fit the model

dc = DummyClassifier(strategy="most_frequent") # initalize the model
dc.fit(ner_X_train, ner_y_train) # fit the model to the training data

DummyClassifier(strategy='most_frequent')

In [24]:
# assess performance using classification_report

states = list(ner_y_train.unique()) # list of unique states in our training dataset

dc_preds = dc.predict(ner_X_test) # make predictions for test data
print(classification_report(y_true = ner_y_test, y_pred = dc_preds, labels = states)) # assess performance by hidden state

              precision    recall  f1-score   support

           O       0.85      1.00      0.92     18417
       B-geo       0.00      0.00      0.00       784
       B-gpe       0.00      0.00      0.00       346
       B-per       0.00      0.00      0.00       410
       I-geo       0.00      0.00      0.00       165
       B-org       0.00      0.00      0.00       363
       I-org       0.00      0.00      0.00       310
       B-tim       0.00      0.00      0.00       390
       B-art       0.00      0.00      0.00         7
       I-art       0.00      0.00      0.00         9
       I-per       0.00      0.00      0.00       420
       I-gpe       0.00      0.00      0.00         4
       I-tim       0.00      0.00      0.00       125
       B-nat       0.00      0.00      0.00         2
       B-eve       0.00      0.00      0.00        13
       I-eve       0.00      0.00      0.00        13
       I-nat       0.00      0.00      0.00         0

   micro avg       0.85   

C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

Overall, we see that this baseline model only performs well for 'O' since it is predicting 'O' for every observation. This shows that we must be extra careful when interpreting the results since the large representation of 'O' in the data will bias the overall results (i.e., the micro/macro/weighted avgs). This is is why we cannot just look at the overall results and must assess performance by hidden state as well.

#### 2.2.2 HMMs
Next we will implement a Hidden Markov Model, or HMM. As review, for HMMs, we have (1) a set of observed states, (2) a set of hidden states, (3) initial state probabilities, (4) state transition probabilities, and (5) observation state probabilities. HMMs are a generative model meaning that we can think of the the hidden states as "generating" the observations. This can help explain why a particular sequence was observed by finding the most likely path that generated the observations. However, it is important to remember that with HMMs, the hidden states are only dependent on the previous state and the observed states are only dependent on its hidden state. 

In [25]:
# source: https://towardsdatascience.com/part-of-speech-tagging-with-hidden-markov-chain-models-e9fccc835c0e
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

In [26]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger") # intialize our model

In [27]:
tags = list(ner_train.Tag) # list of tags
words = list(ner_train.Word) # list of words
words = [word.lower() for word in words] # lowercase all words

tags_count = Counter(tags) # frequency of each tag

tag_combos = [(tags[i],tags[i+1]) for i in range(0,len(tags)-2,2)] # look at each pair of tags
tag_bigrams = Counter(tag_combos) # count the frequency of each pair of tags

In [28]:
starts_tag = ner_train.drop_duplicates(subset = 'Sentence #', keep="first").Tag # starting tags
starting_tag_count = Counter(starts_tag) # frequency of each starting tag

end_tag = ner_train.drop_duplicates(subset = 'Sentence #', keep="last").Tag # ending tags
ending_tag_count = Counter(end_tag) # frequency of each ending tag

In [29]:
# function to calculate the frequency of each tag/word combo

def pair_counts(tags, words):
    counts = defaultdict(lambda: defaultdict(int))
    for tag, word in zip(tags, words):
        counts[tag][word] += 1
    return counts

tag_words_count = pair_counts(tags, words) 

In [30]:
# probability distribution of words at each state

test_words = list(ner_test.Word) # list of test words
test_words = [word.lower() for word in test_words] # lowercase all words
all_words = set(words + test_words) # unique set of all words in train & test combined

to_pass_states = []
for tag, words_dict in tag_words_count.items():
    total = float(sum(words_dict.values()))
    distribution = {word: count/total for word, count in words_dict.items()}
    
    # add default probability for all words (in train and/or test) to prevent errors during prediction
    for word in all_words:
        if word not in list(distribution.keys()):
                distribution[word] = 1/1000
    
    tag_emissions = DiscreteDistribution(distribution)
    tag_state = State(tag_emissions, name=tag)
    to_pass_states.append(tag_state)

In [31]:
basic_model.add_states()

# start probability for each state
start_prob={}
for tag in tags:
    start_prob[tag] = starting_tag_count[tag]/tags_count[tag]
    
for tag_state in to_pass_states:
    basic_model.add_transition(basic_model.start,tag_state,start_prob[tag_state.name])

# end probability for each state
end_prob={}
for tag in tags:
    end_prob[tag]=ending_tag_count[tag]/tags_count[tag]
    
for tag_state in to_pass_states:
    basic_model.add_transition(tag_state,basic_model.end,end_prob[tag_state.name])

In [32]:
# transition probabilities between states

transition_prob_pair = {}
for key in tag_bigrams.keys():
    transition_prob_pair[key] = tag_bigrams.get(key)/tags_count[key[0]]
    
for tag_state in to_pass_states :
    for next_tag_state in to_pass_states:
        try:
            transition = transition_prob_pair[(tag_state.name,next_tag_state.name)]
        except:
            transition = 0 # if transition is not found in data
        basic_model.add_transition(tag_state,next_tag_state,transition)
        
basic_model.bake() # finalize model

Now that we have created the model, we can use it to make predictions on the test data. To do so, we will create a nested list of our test sentences where each sentence is its own list. Then, we will make our predictions on each sentence individually.

In [33]:
# get list of test sentences

sentences_dict_test = {}

for index, row in ner_test.iterrows():
    if row['Sentence #'] not in sentences_dict_test:
        sentences_dict_test[row['Sentence #']] = []
    sentences_dict_test[row['Sentence #']].append((row['Word'].lower()))
    
sentences_test = list(sentences_dict_test.values())
print(sentences_test[1]) # check

['on', 'wednesday', ',', 'police', 're-opened', 'a', 'central', 'square', 'in', 'the', 'city', 'after', 'clearing', 'out', 'protesters', '.']


In [34]:
# make predictions by the sentence

hmm_preds = []
for sentence in sentences_test:
    _, state_path = basic_model.viterbi(sentence)
    tags = [state[1].name for state in state_path[1:-1]]
    hmm_preds.append(tags)  

In [37]:
# assess performance

states = list(ner_y_train.unique()) # list of unique states in our dataset
hmm_preds_concat = [item for sublist in hmm_preds for item in sublist]

print(classification_report(y_true = ner_y_test, y_pred = hmm_preds_concat, labels = states))

              precision    recall  f1-score   support

           O       0.90      0.80      0.85     18417
       B-geo       0.32      0.23      0.27       784
       B-gpe       0.29      0.36      0.32       346
       B-per       0.26      0.47      0.33       410
       I-geo       0.53      0.42      0.47       165
       B-org       0.20      0.26      0.23       363
       I-org       0.11      0.50      0.19       310
       B-tim       0.79      0.53      0.63       390
       B-art       0.00      0.00      0.00         7
       I-art       0.00      0.00      0.00         9
       I-per       0.18      0.46      0.26       420
       I-gpe       0.00      0.00      0.00         4
       I-tim       0.19      0.42      0.26       125
       B-nat       0.00      0.00      0.00         2
       B-eve       0.40      0.31      0.35        13
       I-eve       0.15      0.31      0.21        13
       I-nat       0.00      0.00      0.00         0

   micro avg       0.74   

C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

We see that the results are better than our baseline model but only slightly. This can be at least partially attributed to the fact that we had to add default probabilities (1/1000 in this case) for a large fraction of the word/tag combinations since they did not appear in our training data. Therefore, in order to improve performance in the future, we would want to train our model on substantially more data so that the training vocabulary is more expansive and representative.

* HMMs is generaltive model
* CRFs is discriminitive model 

#### 2.2.3 CRFs
Lastly, we will implement a Conditional Random Field, or CRF for short. Contrary to HMMs, CRFs are a discriminative model with more relaxed assumptions of the features since we do not need to model the joint probability, P(X,Y) where X=states and y=labels. This means we can utilize more features in our models including past and future observations.

In [38]:
# transform training data into list of sentences with each row as a tuple of the word, pos and tag

sentences_dict_train = {}

for index, row in ner_train.iterrows():
    if row['Sentence #'] not in sentences_dict_train:
        sentences_dict_train[row['Sentence #']] = []
    sentences_dict_train[row['Sentence #']].append((row['Word'], row['POS'], row['Tag']))
    
sentences_train = sentences_dict_train.values()

In [39]:
# feature extraction

# source: https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    # CRF can take featrues in: i.e. consider the lower case and upper case 
    # (which is needed for some Captial-meanningful cases)
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

In [40]:
X_train = [sent2features(s) for s in sentences_train] # converting training features
y_train = [sent2labels(s) for s in sentences_train] # converting tags

print(len(X_train)) # check
print(len(y_train)) # check

2000
2000


In [41]:
# fit the model
crf = sklearn_crfsuite.CRF() # initalize the model

# 
try:
    crf.fit(X_train, y_train) # fit the model to the training data
except:
    pass

In [42]:
# transform testing data into list of sentences with each row as a tuple of the word, pos and tag

sentences_dict_test = {}

for index, row in ner_test.iterrows():
    if row['Sentence #'] not in sentences_dict_test:
        sentences_dict_test[row['Sentence #']] = []
    sentences_dict_test[row['Sentence #']].append((row['Word'], row['POS'], row['Tag']))
    
sentences_test = sentences_dict_test.values()

X_test = [sent2features(s) for s in sentences_test]
y_test = [sent2labels(s) for s in sentences_test]

print(len(X_test)) # check
print(len(y_test)) # check

1000
1000


In [43]:
# make predictions & assess performance

crf_preds = crf.predict(X_test) 
crf_preds = [item for sublist in crf_preds for item in sublist] # flatten out predictions

print(classification_report(y_true = ner_y_test, y_pred = crf_preds, labels = states))

              precision    recall  f1-score   support

           O       0.99      0.99      0.99     18417
       B-geo       0.77      0.80      0.78       784
       B-gpe       0.81      0.85      0.83       346
       B-per       0.78      0.75      0.77       410
       I-geo       0.75      0.59      0.66       165
       B-org       0.64      0.60      0.61       363
       I-org       0.71      0.64      0.67       310
       B-tim       0.90      0.73      0.81       390
       B-art       0.00      0.00      0.00         7
       I-art       0.00      0.00      0.00         9
       I-per       0.75      0.95      0.83       420
       I-gpe       0.00      0.00      0.00         4
       I-tim       0.89      0.50      0.64       125
       B-nat       0.00      0.00      0.00         2
       B-eve       1.00      0.31      0.47        13
       I-eve       1.00      0.31      0.47        13
       I-nat       0.00      0.00      0.00         0

   micro avg       0.95   

C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

We see that the CRF model performs the best! Let's see if we can improve performance even more.

* HMM can know what happens in the backscence
* CRFs only do the better clasification, 各有长处

In [44]:
### YOUR CODE: play around with the CRF features/parameters - can you improve performance even more?

### For inspiration, check out https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#let-s-use-conll-2002-data-to-build-a-ner-system

In [45]:
# example model parameters (try different for differnt ideas and context)

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=200,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False)